In [25]:
import requests as r

In [26]:
#get request to MBTA api given a string path
def make_api_call(path):
    url = 'https://api-v3.mbta.com/' + path
    response = r.get(url)
    if response.status_code != r.codes.ok:
        raise RuntimeError(response.status_code, response.reason)
    json = response.json()
    return json['data']



def get_route_long_names():
    data = make_api_call('routes?filter[type]=0,1')
    long_names =[]
    for route in data:
        long_names.append(route['attributes']['long_name'])


    return long_names
def get_route_ids():
    data = make_api_call('routes?filter[type]=0,1')
    ids = []
    for route in data:
        n = route['id']
        ids.append(str(n)) 

    return ids
    

In [27]:
get_route_long_names()

['Red Line',
 'Mattapan Trolley',
 'Orange Line',
 'Green Line B',
 'Green Line C',
 'Green Line D',
 'Green Line E',
 'Blue Line']

In [28]:
class Route():
    def __init__(self, r_id, long_name):
        self.r_id = r_id
        self.long_name=long_name
        self.stops = []
       
    def Route(self, r_id, long_name, stops):
        self.r_id = r_id
        self.long_name=long_name
        self.stops = stops
    def get_id():
        return self.r_id
    def get_long_name():
        return self.long_name
    def get_stops():
        return self.stops
    def add_stop(self, stop):
        self.stops.append(stop)

#assuming these go in main? 
def create_routes():
    ids = get_route_ids()
    long_names = get_route_long_names()
    route_l =[]
    for r_id, long_name in zip(ids, long_names):
        r = Route(r_id,long_name)
        route_l.append(r)
    return route_l
def add_stops(routes):
    for route in routes:    
        r_id = route.r_id
        path = 'stops?filter[route]={}'.format(r_id)
        data = make_api_call(path)
        for stop in data:
            val = stop['attributes']['name']
            route.add_stop(val)

    return routes

In [29]:
get_route_long_names()
get_route_ids()

['Red',
 'Mattapan',
 'Orange',
 'Green-B',
 'Green-C',
 'Green-D',
 'Green-E',
 'Blue']

In [30]:
def get_route_max_stops(routes):
    max_stops = 0
    long_name = ''
    for route in routes:
        num_stops = len(route.stops)
        if num_stops > max_stops:
            max_stops = num_stops
            long_name= route.long_name
    return long_name, max_stops


In [31]:
routes = create_routes()
routes = add_stops(routes)
get_route_max_stops(routes)

('Green Line D', 25)

In [32]:
def get_route_min_stops(routes):
    min_stops = 10000
    long_name = ''
    for route in routes:
        num_stops = len(route.stops)
        if num_stops < min_stops:
            min_stops = num_stops
            long_name= route.long_name
    return long_name, min_stops

In [33]:
get_route_min_stops(routes)

('Mattapan Trolley', 8)

In [40]:
def stop_to_route(routes):
    stop_to_route = {}
    for route in routes:
        for stop in route.stops:
            if stop not in stop_to_route:
                add = []
            else: 
                add = stop_to_route[stop]
            add.append(route)
            stop_to_route[stop] =add
    return stop_to_route
    
def shared_stops(routes):
    stops_routes = stop_to_route(routes)
    shared_stops ={}       
    for stop in stops_routes:
        if len(stops_routes[stop]) >1:
            val = list(map(lambda x:x.long_name,stops_routes[stop] ))            
            shared_stops[stop] = val
    return shared_stops
            
                    
            

In [41]:
shared_stops(routes)

{'Park Street': ['Red Line',
  'Green Line B',
  'Green Line C',
  'Green Line D',
  'Green Line E'],
 'Downtown Crossing': ['Red Line', 'Orange Line'],
 'Ashmont': ['Red Line', 'Mattapan Trolley'],
 'State': ['Orange Line', 'Blue Line'],
 'Haymarket': ['Orange Line', 'Green Line D', 'Green Line E'],
 'North Station': ['Orange Line', 'Green Line D', 'Green Line E'],
 'Government Center': ['Green Line B',
  'Green Line C',
  'Green Line D',
  'Green Line E',
  'Blue Line'],
 'Boylston': ['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E'],
 'Arlington': ['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E'],
 'Copley': ['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E'],
 'Hynes Convention Center': ['Green Line B', 'Green Line C', 'Green Line D'],
 'Kenmore': ['Green Line B', 'Green Line C', 'Green Line D'],
 'Science Park/West End': ['Green Line D', 'Green Line E'],
 'Lechmere': ['Green Line D', 'Green Line E']}

In [49]:

def find_route(stop1,stop2,routes):
    
    
    #maps each stop to avail routes
    stops_w_routes = stop_to_route(routes)
    
    #routes stop2 is on
    stop2_routes = stops_w_routes[stop2]
    
    #stops with multiple routes, mapped to their routes
    nodes = shared_stops(routes)

    visited_routes =[]
    Q =[]
    Q.append((stop1,''))
    while len(Q) !=0:
        n,rt = Q.pop(0)
        
        #routes accessible by this node n
        routes_to_check= stops_w_routes[n]
        
        #check each route for stop2 if they havent been checked yet
        for current_route in routes_to_check:
            if current_route.long_name not in visited_routes:
                visited_routes.append(current_route.long_name )
                if rt != '':
                    
                    rt = rt + ', '+ current_route.long_name
                else:
                    rt = current_route.long_name
                
                #if find it, return it
                if stop2 in current_route.stops:
                    return rt
                #dont find it, mark route as visited
                else:
                    visited_routes.append(current_route.long_name)
                #check each route for the stops that have access to other routes
                for s in current_route.stops:
                    if s in nodes:
                        Q.append((s,rt))
            
                
                
            
            
        
        
find_route('Wonderland','Northeastern University',routes)        

'Blue Line, Green Line B, Green Line C, Green Line D, Green Line E'

In [52]:
def main():
    routes = create_routes()
    routes = add_stops(routes)
    
    #Question 1:
    print(get_route_long_names())
    
    #Question 2: 
    #2.1
    print(get_route_max_stops(routes))
    #2.2
    print(get_route_min_stops(routes))
    #2.3
    print(shared_stops(routes))
    
    
    #Question 3:
    print(find_route('Wonderland','Northeastern University',routes))
    
    
if __name__ == "__main__":
    main()

['Red Line', 'Mattapan Trolley', 'Orange Line', 'Green Line B', 'Green Line C', 'Green Line D', 'Green Line E', 'Blue Line']
('Green Line D', 25)
('Mattapan Trolley', 8)
{'Park Street': ['Red Line', 'Green Line B', 'Green Line C', 'Green Line D', 'Green Line E'], 'Downtown Crossing': ['Red Line', 'Orange Line'], 'Ashmont': ['Red Line', 'Mattapan Trolley'], 'State': ['Orange Line', 'Blue Line'], 'Haymarket': ['Orange Line', 'Green Line D', 'Green Line E'], 'North Station': ['Orange Line', 'Green Line D', 'Green Line E'], 'Government Center': ['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E', 'Blue Line'], 'Boylston': ['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E'], 'Arlington': ['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E'], 'Copley': ['Green Line B', 'Green Line C', 'Green Line D', 'Green Line E'], 'Hynes Convention Center': ['Green Line B', 'Green Line C', 'Green Line D'], 'Kenmore': ['Green Line B', 'Green Line C', 'Green Line D'], 'Sc

KeyboardInterrupt: 